In [ ]:
# --------------------------
# Fix truncated images
# --------------------------
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --------------------------
# Imports
# --------------------------
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from sklearn.utils import resample
from datetime import datetime
from tensorflow.keras import mixed_precision

# --------------------------
# Mixed precision for speed
# --------------------------
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# --------------------------
# Paths (nested dataset)
# --------------------------
base_dir = "/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification"

train_dir = os.path.join(base_dir, "train")
val_dir   = os.path.join(base_dir, "val")
test_dir  = os.path.join(base_dir, "test")

print("Train classes:", os.listdir(train_dir))
print("Val classes:", os.listdir(val_dir))
print("Test classes:", os.listdir(test_dir))

# --------------------------
# Balance training dataset by oversampling
# --------------------------
def balance_dataset(directory):
    classes = os.listdir(directory)
    class_files = {cls: [os.path.join(directory, cls, f) for f in os.listdir(os.path.join(directory, cls))] for cls in classes}
    max_count = max(len(files) for files in class_files.values())
    balanced_files = []
    for cls, files in class_files.items():
        if len(files) < max_count:
            files = resample(files, replace=True, n_samples=max_count, random_state=42)
        balanced_files.extend(files)
    return balanced_files, classes

balanced_train_files, train_classes = balance_dataset(train_dir)
print("Balanced training files per class:", {cls: len([f for f in balanced_train_files if cls in f]) for cls in train_classes})

# --------------------------
# Parameters
# --------------------------
img_size = (160, 160)  # smaller for faster training
batch_size = 32        # larger batch for speed

# --------------------------
# Data Generators
# --------------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2]
)

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    train_dir, target_size=img_size, batch_size=batch_size, class_mode='binary'
)

val_gen = val_test_datagen.flow_from_directory(
    val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary'
)

test_gen = val_test_datagen.flow_from_directory(
    test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False
)

# --------------------------
# DenseNet121 Model
# --------------------------
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_size+(3,))
base_model.trainable = False  # freeze all first

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid', dtype='float32')  # force float32
])

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# --------------------------
# Callbacks
# --------------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/fracture_model_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# --------------------------
# Train Phase 1: Frozen base
# --------------------------
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=7,
    steps_per_epoch=len(train_gen)//2,
    validation_steps=len(val_gen)//2,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Fine-tune last DenseNet layers
# --------------------------
base_model.trainable = True
for layer in base_model.layers[:-50]:  # only fine-tune last 50 layers
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen)//2,
    validation_steps=len(val_gen)//2,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Save final model with timestamp
# --------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_name = f"/kaggle/working/fracture_model_final_{timestamp}.keras"
model.save(final_model_name)
print(f"Final model saved as {final_model_name}")

# --------------------------
# Evaluate on test set
# --------------------------
loss, acc = model.evaluate(test_gen)
print(f"\nFinal Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}")


In [ ]:
# --------------------------
# Fix truncated images
# --------------------------
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --------------------------
# Imports
# --------------------------
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from sklearn.utils import shuffle
from datetime import datetime
from tensorflow.keras import mixed_precision
import shutil
from pathlib import Path

# --------------------------
# Mixed precision for speed
# --------------------------
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# --------------------------
# Paths
# --------------------------
base_dir = Path("/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification")

train_dir = base_dir / "train"
val_dir   = base_dir / "val"
test_dir  = base_dir / "test"

# --------------------------
# Create truly balanced training dataset
# --------------------------
classes = ["Fractured", "Non-Fractured"]

# Count images
counts = {cls: len(list((train_dir/cls).glob("*"))) for cls in classes}
min_count = min(counts.values())

# Balanced training directory
balanced_train_dir = Path("/kaggle/working/train_balanced")
if balanced_train_dir.exists():
    shutil.rmtree(balanced_train_dir)
balanced_train_dir.mkdir()

for cls in classes:
    (balanced_train_dir/cls).mkdir(parents=True, exist_ok=True)
    files = list((train_dir/cls).glob("*"))
    sampled_files = np.random.choice(files, min_count, replace=False)
    for f in sampled_files:
        shutil.copy(f, balanced_train_dir/cls/f.name)

print("Balanced dataset created:")
for cls in classes:
    print(cls, len(list((balanced_train_dir/cls).glob("*"))))

# --------------------------
# Parameters
# --------------------------
img_size = (160, 160)
batch_size = 32

# --------------------------
# Data Generators
# --------------------------
train_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input,
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.2,
    horizontal_flip=True,
    brightness_range=[0.8,1.2]
)

val_test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_gen = train_datagen.flow_from_directory(
    balanced_train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=True
)

val_gen = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary'
)

test_gen = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='binary',
    shuffle=False
)

# --------------------------
# DenseNet121 Model
# --------------------------
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_size+(3,))
base_model.trainable = False  # freeze all first

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid', dtype='float32')  # force float32
])

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# --------------------------
# Callbacks
# --------------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/fracture_model_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# --------------------------
# Train Phase 1: Frozen base
# --------------------------
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=7,
    steps_per_epoch=len(train_gen)//2,
    validation_steps=len(val_gen)//2,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Fine-tune last DenseNet layers
# --------------------------
base_model.trainable = True
for layer in base_model.layers[:-50]:  # fine-tune last 50 layers
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    epochs=10,
    steps_per_epoch=len(train_gen)//2,
    validation_steps=len(val_gen)//2,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Save final model with timestamp
# --------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_name = f"/kaggle/working/fracture_model_final_{timestamp}.keras"
model.save(final_model_name)
print(f"Final model saved as {final_model_name}")

# --------------------------
# Evaluate on test set
# --------------------------
loss, acc = model.evaluate(test_gen)
print(f"\nFinal Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}")


In [ ]:
# --------------------------
# Fix truncated images
# --------------------------
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --------------------------
# Imports
# --------------------------
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from sklearn.utils import shuffle
from datetime import datetime
from tensorflow.keras import mixed_precision

# --------------------------
# Mixed precision
# --------------------------
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# --------------------------
# Paths
# --------------------------
train_dir = "/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train"
val_dir   = "/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val"
test_dir  = "/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test"

classes = ["Fractured", "Non-Fractured"]
img_size = (160, 160)
batch_size = 32

# --------------------------
# Prepare true balanced training generator
# --------------------------
file_paths = {cls: [os.path.join(train_dir, cls, f) for f in os.listdir(os.path.join(train_dir, cls))] for cls in classes}
max_count = max(len(file_paths[cls]) for cls in classes)

# Oversample minority class
for cls in classes:
    n = len(file_paths[cls])
    if n < max_count:
        extra = np.random.choice(file_paths[cls], max_count - n, replace=True)
        file_paths[cls].extend(extra)

# Flatten and shuffle
all_files = []
all_labels = []
for i, cls in enumerate(classes):
    all_files.extend(file_paths[cls])
    all_labels.extend([i] * len(file_paths[cls]))
all_files, all_labels = shuffle(all_files, all_labels, random_state=42)

# Generator
def balanced_generator(file_paths, labels, batch_size):
    while True:
        for i in range(0, len(file_paths), batch_size):
            batch_files = file_paths[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            batch_images = []
            for f in batch_files:
                img = tf.keras.preprocessing.image.load_img(f, target_size=img_size)
                x = tf.keras.preprocessing.image.img_to_array(img)
                x = preprocess_input(x)
                batch_images.append(x)
            yield np.array(batch_images), np.array(batch_labels)

train_gen = balanced_generator(all_files, all_labels, batch_size)
steps_per_epoch = len(all_files) // batch_size

# --------------------------
# Validation & test generators
# --------------------------
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
test_gen = val_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False)

# --------------------------
# DenseNet121 model
# --------------------------
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_size+(3,))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid', dtype='float32')  # float32 for mixed precision
])

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# --------------------------
# Callbacks
# --------------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/fracture_model_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# --------------------------
# Train Phase 1: Frozen base
# --------------------------
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=len(val_gen),
    epochs=7,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Fine-tune last 50 layers
# --------------------------
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=len(val_gen),
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Save final model with timestamp
# --------------------------
from datetime import datetime
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_name = f"/kaggle/working/fracture_model_final_{timestamp}.keras"
model.save(final_model_name)
print(f"Final model saved as {final_model_name}")

# --------------------------
# Evaluate on test set
# --------------------------
loss, acc = model.evaluate(test_gen)
print(f"\nFinal Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}")


In [ ]:
import os

train_dir = '/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train'
val_dir = '/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val'
test_dir = '/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test'

# Check if the directories exist
for directory in [train_dir, val_dir, test_dir]:
    if not os.path.exists(directory):
        print(f"Directory does not exist: {directory}")
    else:
        print(f"Directory exists: {directory}")
        # List subdirectories
        subdirs = os.listdir(directory)
        print(f"Subdirectories in {directory}: {subdirs}")



In [ ]:
# --------------------------
# Fix truncated images
# --------------------------
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

# --------------------------
# Imports
# --------------------------
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.densenet import DenseNet121, preprocess_input
from sklearn.utils import shuffle
from datetime import datetime
from tensorflow.keras import mixed_precision

# --------------------------
# Mixed precision
# --------------------------
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

# --------------------------
# Paths
# --------------------------
train_dir = "/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/train"
val_dir   = "/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/val"
test_dir  = "/kaggle/input/fracture-multi-region-x-ray-data/Bone_Fracture_Binary_Classification/Bone_Fracture_Binary_Classification/test"

# --------------------------
# Parameters
# --------------------------
classes = ["fractured", "not fractured"]  # EXACT folder names
img_size = (160, 160)
batch_size = 32

# --------------------------
# Prepare true balanced training generator
# --------------------------
file_paths = {cls: [os.path.join(train_dir, cls, f) for f in os.listdir(os.path.join(train_dir, cls))] for cls in classes}
max_count = max(len(file_paths[cls]) for cls in classes)

# Oversample minority class
for cls in classes:
    n = len(file_paths[cls])
    if n < max_count:
        extra = np.random.choice(file_paths[cls], max_count - n, replace=True)
        file_paths[cls].extend(extra)

# Flatten and shuffle
all_files = []
all_labels = []
for i, cls in enumerate(classes):
    all_files.extend(file_paths[cls])
    all_labels.extend([i] * len(file_paths[cls]))
all_files, all_labels = shuffle(all_files, all_labels, random_state=42)

# Generator
def balanced_generator(file_paths, labels, batch_size):
    while True:
        for i in range(0, len(file_paths), batch_size):
            batch_files = file_paths[i:i+batch_size]
            batch_labels = labels[i:i+batch_size]
            batch_images = []
            for f in batch_files:
                img = tf.keras.preprocessing.image.load_img(f, target_size=img_size)
                x = tf.keras.preprocessing.image.img_to_array(img)
                x = preprocess_input(x)
                batch_images.append(x)
            yield np.array(batch_images), np.array(batch_labels)

train_gen = balanced_generator(all_files, all_labels, batch_size)
steps_per_epoch = len(all_files) // batch_size

# --------------------------
# Validation & test generators
# --------------------------
val_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function=preprocess_input)
val_gen = val_datagen.flow_from_directory(val_dir, target_size=img_size, batch_size=batch_size, class_mode='binary')
test_gen = val_datagen.flow_from_directory(test_dir, target_size=img_size, batch_size=batch_size, class_mode='binary', shuffle=False)

# --------------------------
# DenseNet121 model
# --------------------------
base_model = DenseNet121(weights='imagenet', include_top=False, input_shape=img_size+(3,))
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dropout(0.4),
    layers.Dense(1, activation='sigmoid', dtype='float32')
])

model.compile(
    optimizer=optimizers.Adam(1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# --------------------------
# Callbacks
# --------------------------
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, min_lr=1e-6, verbose=1),
    tf.keras.callbacks.ModelCheckpoint(
        filepath='/kaggle/working/fracture_model_best.keras',
        monitor='val_accuracy',
        save_best_only=True,
        verbose=1
    )
]

# --------------------------
# Train Phase 1: Frozen base
# --------------------------
history1 = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=len(val_gen),
    epochs=7,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Fine-tune last 50 layers
# --------------------------
base_model.trainable = True
for layer in base_model.layers[:-50]:
    layer.trainable = False

model.compile(
    optimizer=optimizers.Adam(1e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history2 = model.fit(
    train_gen,
    validation_data=val_gen,
    steps_per_epoch=steps_per_epoch,
    validation_steps=len(val_gen),
    epochs=10,
    callbacks=callbacks,
    verbose=1
)

# --------------------------
# Save final model with timestamp
# --------------------------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
final_model_name = f"/kaggle/working/fracture_model_final_{timestamp}.keras"
model.save(final_model_name)
print(f"Final model saved as {final_model_name}")

# --------------------------
# Evaluate on test set
# --------------------------
loss, acc = model.evaluate(test_gen)
print(f"\nFinal Test Accuracy: {acc:.4f}, Test Loss: {loss:.4f}")

